In [1]:
from mindsponge import Sponge
from mindsponge import ForceField
from mindsponge.optimizer import SteepestDescent
from mindsponge.control import VelocityVerlet
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.control import Langevin
from mindsponge import set_global_units
from mindsponge import Protein
from mindsponge import UpdaterMD
from mindsponge.function import VelocityGenerator
from mindspore import context

In [2]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")

## 单位设置

这里我们提到的全局单位设置，主要是指输入输出的长度单位和能量单位，在MindSPONGE的计算过程中会自动根据默认单位进行计算，再将返回的结果转换成用户定义的单位。常用的长度单位是nm和A，一般在PDB文件里面是用A为单位。能量单位比较常用的是kJ/mol和kcal/mol。

In [3]:
set_global_units('nm', 'kj/mol')

## 蛋白质案例

我们提供了一些简单的蛋白质案例以供测试，一般用pdb(Protein Data Bank)格式来存储蛋白质文件。下述案例就是一个不含H原子的pdb文件（该构象来自于MEGAProtein的预测结果）。

![](../../docs/case2-1.png)

在使用MindSPONGE构建Protein对象时，会自动为其在相对合理的位置补充H原子（如下图所示）。而这一过程之后，最好对其进行一次能量最小化的操作，以确保得到的构象中不会存在太多的冲突，比如两个原子距离太近等。

![](../../docs/case2-3.png)

如果使用常用的蛋白质可视化算法NewCartoon来展示的话，可以更清晰的看到这个构象中所存在的$\alpha$螺旋和$\beta$折叠等区域：

![](../../docs/case2-2.png)

In [4]:
pdb_name = 'case2.pdb'
system = Protein(pdb=pdb_name, rebuild_hydrogen=True)

1 H-Adding task complete.


In [5]:
energy = ForceField(system, 'AMBER.FF14SB')

In [6]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)

In [7]:
md = Sponge(system, energy, min_opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [8]:
run_info = RunInfo(10)
md.run(500, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-03-25 17:06:51
[MindSPONGE] Step: 0, E_pot: 141277.78
[MindSPONGE] Step: 10, E_pot: 96736.516
[MindSPONGE] Step: 20, E_pot: 75656.484
[MindSPONGE] Step: 30, E_pot: 62071.25
[MindSPONGE] Step: 40, E_pot: 52376.703
[MindSPONGE] Step: 50, E_pot: 45027.332
[MindSPONGE] Step: 60, E_pot: 39223.36
[MindSPONGE] Step: 70, E_pot: 34505.496
[MindSPONGE] Step: 80, E_pot: 30588.41
[MindSPONGE] Step: 90, E_pot: 27282.955
[MindSPONGE] Step: 100, E_pot: 24456.77
[MindSPONGE] Step: 110, E_pot: 22013.086
[MindSPONGE] Step: 120, E_pot: 19879.0
[MindSPONGE] Step: 130, E_pot: 17998.316
[MindSPONGE] Step: 140, E_pot: 16327.084
[MindSPONGE] Step: 150, E_pot: 14830.459
[MindSPONGE] Step: 160, E_pot: 13480.793
[MindSPONGE] Step: 170, E_pot: 12255.748
[MindSPONGE] Step: 180, E_pot: 11137.307
[MindSPONGE] Step: 190, E_pot: 10110.74
[MindSPONGE] Step: 200, E_pot: 9163.99
[MindSPONGE] Step: 210, E_pot: 8287.027
[MindSPONGE] Step: 220, E_pot: 7471.5547
[MindSPONGE] Step: 230,

In [9]:
vgen = VelocityGenerator(300)
velocity = vgen(system.coordinate.shape, system.atom_mass)

In [10]:
opt = UpdaterMD(system,
                integrator=VelocityVerlet(system),
                thermostat=Langevin(system, 300),
                time_step=1e-3,
                velocity=velocity)

In [11]:
md = Sponge(system, energy, min_opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b06.h5md', save_freq=10, write_velocity=True, write_force=True)

In [13]:
md.change_optimizer(opt)
md.run(2000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-03-25 17:07:53
[MindSPONGE] Step: 0, E_pot: -4076.3872, E_kin: 9260.689, E_tot: 5184.3022, Temperature: 293.72495
[MindSPONGE] Step: 10, E_pot: -3176.852, E_kin: 8410.569, E_tot: 5233.7173, Temperature: 266.76132
[MindSPONGE] Step: 20, E_pot: -3971.753, E_kin: 9656.892, E_tot: 5685.1387, Temperature: 306.29144
[MindSPONGE] Step: 30, E_pot: -3090.626, E_kin: 8912.391, E_tot: 5821.7646, Temperature: 282.6778
[MindSPONGE] Step: 40, E_pot: -4246.1826, E_kin: 10109.949, E_tot: 5863.7666, Temperature: 320.66122
[MindSPONGE] Step: 50, E_pot: -4114.4224, E_kin: 9827.102, E_tot: 5712.679, Temperature: 311.69006
[MindSPONGE] Step: 60, E_pot: -4198.6094, E_kin: 9763.93, E_tot: 5565.3203, Temperature: 309.68643
[MindSPONGE] Step: 70, E_pot: -4347.147, E_kin: 9700.381, E_tot: 5353.234, Temperature: 307.6708
[MindSPONGE] Step: 80, E_pot: -4473.815, E_kin: 9753.679, E_tot: 5279.864, Temperature: 309.36127
[MindSPONGE] Step: 90, E_pot: -4314.6777, E_kin: 9675.14